In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

functions define

In [2]:
def get_data_dict(data):
    data_dict = {}
    for m in range(12):
        row = m*18*20
        temp = data.iloc[row:row+18]
        for d in range(1,20):
            row += 18
            temp = pd.concat([temp,data.iloc[row:row+18]],axis=1)
        temp.set_axis(range(24*20),axis=1,inplace=True)
        data_dict[m] = temp
    return data_dict

def getMonthTrainData(month,hours,features):
    label = month.loc['PM2.5',hours:]
    X = np.empty((480-hours,len(features)*hours))
    for i in range(480-hours):
        X[i,:] = month.loc[features,i:i+hours-1].values.flatten()
    return X,label.values.astype(np.float)

def getYearTrainData(dict,hours,features):
    X,y = getMonthTrainData(dict[0],hours,features)
    for m in range(1,12):
        temp_X,temp_y = getMonthTrainData(dict[m],hours,features)
        X = np.concatenate((X,temp_X),axis=0)
        y = np.concatenate((y,temp_y),axis=0)
    return X,y.reshape(y.shape[0],-1)

def normal(X):
    mu = X.mean(axis=0)
    sigma = X.std(axis=0)
    return (X-mu)/sigma,mu,sigma

读取训练数据

In [3]:
data = pd.read_csv('train.csv',engine='python')

data.columns = ['Date', 'Place', 'Test', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23']

data.drop(['Date','Place'],axis='columns',inplace=True)

data.set_index('Test',inplace=True)

data[data=='NR'] = 0

data_dict = get_data_dict(data)

In [40]:
hours = 9
features = ['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx', 'O3', 'PM10',
       'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC', 'WD_HR', 'WIND_DIREC',
       'WIND_SPEED', 'WS_HR']

In [52]:
X,y = getYearTrainData(data_dict,hours,features)

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [53]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [54]:
def loss_fun(theta,X,y):
    m = y.shape[0]
    y_pred = X@theta
    temp = y_pred - y
    loss = temp.T@temp/m/2
    return loss.item()

In [55]:
loss_fun(model.coef_.T,X_train,y_train),loss_fun(model.coef_.T,X_test,y_test)

(18.169186806949774, 22.830979494955102)

In [56]:
model.coef_.shape

(1, 162)